# Web Crawler

In [1]:
import math
import time

# A compendium of commonly-used regular expressions.
import re

# The fundamental package for scientific computing with Python.
import numpy as np

# Flexible and powerful data analysis / manipulation library for Python, providing labeled data structures similar to R data.frame objects, statistical functions, and much more
import pandas as pd

In [2]:
# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm

# A simple, yet elegant HTTP library.
import requests

# A browser automation framework and ecosystem.
from selenium import webdriver
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Beautiful Soup is a library that makes it easy to scrape information from web pages.
from bs4 import BeautifulSoup

In [3]:
# The old socket.ssl() support for TLS over sockets is being superseded in Python 2.6 by a new ‘ssl’ module. This package brings that module to older Python releases, 2.3.5 and up (it may also work on older versions of 2.3, but we haven’t tried it).
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
driver =webdriver.Chrome('chromedriver',options=options)

In [4]:


params = {
    'section': 'qna', # kin, qna, ency
    'period': '1w', # 1w, 1m, 2002.01.01.%7C2020.12.09.
    'page': 1,
    'query': '여자친구+선물',
}
params['query'] = re.sub(' ', '+', params['query'])

endpoint = 'https://kin.naver.com/search/list.nhn?section={}&period={}&page={}&query={}'\
    .format(params['section'], params['period'], params['page'], params['query'])

driver.get(endpoint)

In [5]:

WebDriverWait(driver, 10)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
print(soup.select_one('h2 span.number'))

<span class="number">(<em>1-10/33</em>)</span>


In [7]:

# paging
page_tag = soup.select_one('h2 span.number').get_text(strip=True)
page_txt = re.sub('\(|\)', '', page_tag).split('/')
num_of_rows = int(page_txt[0].split('-')[1])
total_count = int(page_txt[1])
total_page = math.ceil(total_count / num_of_rows)

In [8]:

# series
dataset = []

for page_no in tqdm(np.arange(1, total_page+1)):
    try:

        params.update({'page': page_no})

        endpoint = 'https://kin.naver.com/search/list.nhn?section={}&period={}&page={}&query={}'\
            .format(params['section'], params['period'], params['page'], params['query'])

        driver.get(endpoint)
        

        # parse
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        menuitem = soup.select('#s_content ul.basic1 li')

        for item in menuitem:
            try:

                element = item.select_one('._searchListTitleAnchor')
                text = element.get_text(strip=True)
                href = element.get('href')

                data = {}
                data.update({ 'title':text, 'href':href })

                for arr in re.sub('.*\?', '', href).split('&'):
                    txt = arr.split('=')
                    data.update({ txt[0]:txt[1] })

                dataset.append(data)

            except Exception as e:
                print(e)

    except Exception as e:
        print(page_no, e)

# quit() is a webdriver command which calls the driver.dispose method, which in turn closes all the browser windows and terminates the WebDriver session. If we do not use quit() at the end of program, the WebDriver session will not be closed properly and the files will not be cleared off memory. This may result in memory leak errors.
driver.quit()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.07it/s]


In [9]:

# dataframe
df1 = pd.DataFrame(dataset)

In [10]:
df1

,title,href,d1id,dirId,docId,qb,enc,section,rank,search_sort,spq
0,저렴한곳 추천좀 (여자친구선물해줄거임),https://kin.naver.com/qna/detail.naver?d1id=5&...,5,5010601,437814039,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,1,0,0
1,여자친구 선물,https://kin.naver.com/qna/detail.naver?d1id=5&...,5,5040302,437181994,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,2,0,0
2,20대여자친구 선물,https://kin.naver.com/qna/detail.naver?d1id=5&...,5,50402,437767955,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,3,0,0
3,여자친구 선물,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,80101,437543960,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,4,0,0
4,여자친구 선물,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,80101,437420908,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,5,0,0
5,"여자친구 선물을 미리 말하게된다면,",https://kin.naver.com/qna/detail.naver?d1id=8&...,8,80101,437789447,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,6,0,0
6,중2여자친구 선물,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,8030205,437591042,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,7,0,0
7,20대 후반 여자 선물 골라주세요!,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,80101,437852283,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,8,0,0
8,나이키 에어포스,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,8040202,437573724,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,9,0,0
9,컨버스 척 70 로우 사이즈 5업 해도...,https://kin.naver.com/qna/detail.naver?d1id=8&...,8,8040202,437702098,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,10,0,0


In [14]:
# the old driver has been quit, so we need to init a new one

from selenium.webdriver.common.by import By

driver =webdriver.Chrome('chromedriver',options=options)
# series
dataset = []

for page_no in tqdm(df1.index):
    try:
        endpoint = df1['href'][page_no]
        
        print(endpoint)
        
        driver.get(endpoint)

        # parse
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # readmore
        num_of_more = int(soup.select_one('#nextPageButton ._currentAnswerCount').get_text(strip=True))
        total_more_count = int(soup.select_one('#nextPageButton ._totalAnswerCount').get_text(strip=True))
        total_more = math.floor(total_more_count / num_of_more)
        
        
        time.sleep(5)
        
        for idx in np.arange(0, total_more):
            driver.find_element(By.CSS_SELECTOR,"#nextPageButton").click()
            time.sleep(1)

        # data
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        menuitem = soup.select('.answer-content__list .answer-content__item')

        for item in menuitem:
            try:

                element = item.select_one('._endContentsText')
                content = element.get_text(strip=True)

                data = {}
                data.update({ 'content':content })

                for arr in re.sub('.*\?', '', endpoint).split('&'):
                    txt = arr.split('=')
                    data.update({ txt[0]:txt[1] })

                dataset.append(data)

            except Exception as e:
                print(e)

    except Exception as e:
        print(page_no, e)

# quit() is a webdriver command which calls the driver.dispose method, which in turn closes all the browser windows and terminates the WebDriver session. If we do not use quit() at the end of program, the WebDriver session will not be closed properly and the files will not be cleared off memory. This may result in memory leak errors.
driver.quit()

# dataframe
df2 = pd.DataFrame(dataset)

  0%|                                                                                                                                                                                          | 0/33 [00:00<?, ?it/s]

https://kin.naver.com/qna/detail.naver?d1id=5&dirId=5010601&docId=437814039&qb=7Jes7J6Q7Lmc6rWsIOyEoOusvA==&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0


  3%|█████▍                                                                                                                                                                            | 1/33 [00:07<04:06,  7.72s/it]

0 Message: element not interactable
  (Session info: headless chrome=109.0.5414.119)
Stacktrace:
0   chromedriver                        0x000000010080f0fc chromedriver + 4223228
1   chromedriver                        0x0000000100797284 chromedriver + 3732100
2   chromedriver                        0x000000010044b448 chromedriver + 275528
3   chromedriver                        0x0000000100488660 chromedriver + 525920
4   chromedriver                        0x000000010047b824 chromedriver + 473124
5   chromedriver                        0x000000010047affc chromedriver + 471036
6   chromedriver                        0x00000001004bf360 chromedriver + 750432
7   chromedriver                        0x0000000100479748 chromedriver + 464712
8   chromedriver                        0x000000010047a7f0 chromedriver + 468976
9   chromedriver                        0x00000001007dfe08 chromedriver + 4029960
10  chromedriver                        0x00000001007e3698 chromedriver + 4044440
11  chro

  6%|██████████▊                                                                                                                                                                       | 2/33 [00:16<04:13,  8.19s/it]

...

 94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 31/33 [03:11<00:13,  6.50s/it]

https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80603&docId=437433089&qb=7Jes7J6Q7Lmc6rWsIOyEoOusvA==&enc=utf8&section=kin.qna&rank=32&search_sort=0&spq=0


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋     | 32/33 [03:17<00:06,  6.33s/it]

https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=437635617&qb=7Jes7J6Q7Lmc6rWsIOyEoOusvA==&enc=utf8&section=kin.qna&rank=33&search_sort=0&spq=0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [03:23<00:00,  6.17s/it]


In [12]:
df2

,content,d1id,dirId,docId,qb,enc,section,rank,search_sort,spq
0,서프라이즈로 하세요그게 감동 두배입니다,8,80101,437789447,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,6,0,0
1,당연히 서프라이즈해야죠.안그럼..그날 또 다른 무언가를 사야해요...ㅎㅎㅎㅎ,8,80101,437789447,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,6,0,0
2,오면 주는게 좋죠 ㅎㅎ 서프라이즈,8,80101,437789447,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,6,0,0
3,안녕하세요!연애 전문 어썸미 상담소입니다 :)​미리 상의하지 않고 준비하신 선물이라...,8,80101,437789447,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,6,0,0
4,중학생입장에서 좀 부담스러울 수도 있을 것 같아요 향수나 핸드크림은 어떤가요,8,8030205,437591042,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,7,0,0
5,안녕하세요​좀 부담스럽게 느낄수 있을꺼 같네요​목걸이를 선물 해보시는건 어떨까요?​...,8,8030205,437591042,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,7,0,0
6,​​​​예쁜 악세사리 선물은 어떠신가요?​​​​준샵 목걸이 2가지 추천합니다.​​​...,8,8030205,437591042,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,7,0,0
7,파이토젠 퀸플러스 좋아요​여자친구가 생리통이 심해서 걱정하며사줬던 영양제 선물인데생...,8,80101,437852283,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,8,0,0
8,같은 제품이지만 하단에 있는 제품명 앞에(W)는 여성용입니다.그러므로 하단의 제품으...,8,8040202,437573724,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,9,0,0
9,넵 컨버스는 다른 신발들 보다 작게 나왔다고 느꼈고​그래서 최소 반업은 하시는 것을...,8,8040202,437702098,7Jes7J6Q7Lmc6rWsIOyEoOusvA,utf8,kin.qna,10,0,0


In [13]:
# export
df = pd.merge(df1, df2, on='docId', how='right')
df = df[['docId', 'title', 'content', 'href']]
df

,docId,title,content,href
0,437789447,"여자친구 선물을 미리 말하게된다면,",서프라이즈로 하세요그게 감동 두배입니다,https://kin.naver.com/qna/detail.naver?d1id=8&...
1,437789447,"여자친구 선물을 미리 말하게된다면,",당연히 서프라이즈해야죠.안그럼..그날 또 다른 무언가를 사야해요...ㅎㅎㅎㅎ,https://kin.naver.com/qna/detail.naver?d1id=8&...
2,437789447,"여자친구 선물을 미리 말하게된다면,",오면 주는게 좋죠 ㅎㅎ 서프라이즈,https://kin.naver.com/qna/detail.naver?d1id=8&...
3,437789447,"여자친구 선물을 미리 말하게된다면,",안녕하세요!연애 전문 어썸미 상담소입니다 :)​미리 상의하지 않고 준비하신 선물이라...,https://kin.naver.com/qna/detail.naver?d1id=8&...
4,437591042,중2여자친구 선물,중학생입장에서 좀 부담스러울 수도 있을 것 같아요 향수나 핸드크림은 어떤가요,https://kin.naver.com/qna/detail.naver?d1id=8&...
5,437591042,중2여자친구 선물,안녕하세요​좀 부담스럽게 느낄수 있을꺼 같네요​목걸이를 선물 해보시는건 어떨까요?​...,https://kin.naver.com/qna/detail.naver?d1id=8&...
6,437591042,중2여자친구 선물,​​​​예쁜 악세사리 선물은 어떠신가요?​​​​준샵 목걸이 2가지 추천합니다.​​​...,https://kin.naver.com/qna/detail.naver?d1id=8&...
7,437852283,20대 후반 여자 선물 골라주세요!,파이토젠 퀸플러스 좋아요​여자친구가 생리통이 심해서 걱정하며사줬던 영양제 선물인데생...,https://kin.naver.com/qna/detail.naver?d1id=8&...
8,437573724,나이키 에어포스,같은 제품이지만 하단에 있는 제품명 앞에(W)는 여성용입니다.그러므로 하단의 제품으...,https://kin.naver.com/qna/detail.naver?d1id=8&...
9,437702098,컨버스 척 70 로우 사이즈 5업 해도...,넵 컨버스는 다른 신발들 보다 작게 나왔다고 느꼈고​그래서 최소 반업은 하시는 것을...,https://kin.naver.com/qna/detail.naver?d1id=8&...
